# Importing necessary libraries

In [5]:
import pandas as pd
import numpy as np
import re
import string
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.corpus import stopwords
import nltk

In [16]:
!pip install tensorflow

In [17]:
!pip install nltk scikit-learn

# Load the dataset

In [2]:
file_path = '/content/Sentiment.csv'
data = pd.read_csv(file_path)

# Display first few rows of the dataset for reference

In [3]:
data.head()

,id,candidate,candidate_confidence,relevant_yn,relevant_yn_confidence,sentiment,sentiment_confidence,subject_matter,subject_matter_confidence,candidate_gold,...,relevant_yn_gold,retweet_count,sentiment_gold,subject_matter_gold,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,1,No candidate mentioned,1.0,yes,1.0,Neutral,0.6578,None of the above,1.0000,NaN,...,NaN,5,NaN,NaN,RT @NancyLeeGrahn: How did everyone feel about...,NaN,2015-08-07 09:54:46 -0700,629697200650592256,NaN,Quito
1,2,Scott Walker,1.0,yes,1.0,Positive,0.6333,None of the above,1.0000,NaN,...,NaN,26,NaN,NaN,RT @ScottWalker: Didn't catch the full #GOPdeb...,NaN,2015-08-07 09:54:46 -0700,629697199560069120,NaN,NaN
2,3,No candidate mentioned,1.0,yes,1.0,Neutral,0.6629,None of the above,0.6629,NaN,...,NaN,27,NaN,NaN,RT @TJMShow: No mention of Tamir Rice and the ...,NaN,2015-08-07 09:54:46 -0700,629697199312482304,NaN,NaN
3,4,No candidate mentioned,1.0,yes,1.0,Positive,1.0000,None of the above,0.7039,NaN,...,NaN,138,NaN,NaN,RT @RobGeorge: That Carly Fiorina is trending ...,NaN,2015-08-07 09:54:45 -0700,629697197118861312,Texas,Central Time (US & Canada)
4,5,Donald Trump,1.0,yes,1.0,Positive,0.7045,None of the above,1.0000,NaN,...,NaN,156,NaN,NaN,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,NaN,2015-08-07 09:54:45 -0700,629697196967903232,NaN,Arizona


In [4]:
data.columns

Index(['id', 'candidate', 'candidate_confidence', 'relevant_yn',
       'relevant_yn_confidence', 'sentiment', 'sentiment_confidence',
       'subject_matter', 'subject_matter_confidence', 'candidate_gold', 'name',
       'relevant_yn_gold', 'retweet_count', 'sentiment_gold',
       'subject_matter_gold', 'text', 'tweet_coord', 'tweet_created',
       'tweet_id', 'tweet_location', 'user_timezone'],
      dtype='object')

# Download NLTK data

In [6]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# Filter relevant columns

In [7]:
data = data[['candidate', 'sentiment', 'text']]

# Preprocessing function

In [8]:
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'\@\w+|\#','', text)  # Remove mentions and hashtags
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = re.sub(r'\d+', '', text)  # Remove numbers
    stop_words = set(stopwords.words('english'))
    text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stop words
    return text

In [9]:
data['cleaned_text'] = data['text'].apply(preprocess_text)

# Label encoding for sentiment

In [10]:
encoder = LabelEncoder()
data['sentiment_label'] = encoder.fit_transform(data['sentiment'])

# Tokenization and padding and  Labels

In [11]:
max_words = 5000
max_len = 100
tokenizer = Tokenizer(num_words=max_words, lower=True)
tokenizer.fit_on_texts(data['cleaned_text'])
sequences = tokenizer.texts_to_sequences(data['cleaned_text'])
X = pad_sequences(sequences, maxlen=max_len)


y = tf.keras.utils.to_categorical(data['sentiment_label'], num_classes=3)

# Train-test split

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model development

In [18]:
model = Sequential([
    Embedding(max_words, 100),  # Removed input_length
    SpatialDropout1D(0.2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

In [19]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model training

In [20]:
history = model.fit(
    X_train, y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.1,
    verbose=1
)

Epoch 1/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 35s 208ms/step - accuracy: 0.6048 - loss: 0.9501 - val_accuracy: 0.6712 - val_loss: 0.7924
Epoch 2/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 43s 219ms/step - accuracy: 0.6898 - loss: 0.7126 - val_accuracy: 0.6541 - val_loss: 0.7856
Epoch 3/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 40s 214ms/step - accuracy: 0.7636 - loss: 0.5825 - val_accuracy: 0.6784 - val_loss: 0.8089
Epoch 4/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 42s 219ms/step - accuracy: 0.7919 - loss: 0.4994 - val_accuracy: 0.6820 - val_loss: 0.8931
Epoch 5/5
157/157 ━━━━━━━━━━━━━━━━━━━━ 41s 217ms/step - accuracy: 0.8186 - loss: 0.4462 - val_accuracy: 0.6486 - val_loss: 0.9862


# Evaluate the model

In [21]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

87/87 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.6602 - loss: 0.9208
Test Accuracy: 65.80%
